In [1]:
import pandas as pd
import json
from openai import OpenAI
import os
from tqdm.notebook import tqdm
from langchain.output_parsers import PydanticOutputParser
from criteria_response_parser import CriteriaResponse
from dotenv import load_dotenv

In [2]:
load_dotenv()

data_for_evaluation = pd.read_csv("dify-17.06.2025.csv")
data_for_evaluation

,pergunta,classe,resposta,response_dify
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...,Olá! Sou o assistente do gov.br e estou aqui p...
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...,Olá! Sou o assistente do gov.br e posso ajudar...
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe...",Olá! Sou o assistente do gov.br e posso te aju...
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...,Olá! Eu sou o assistente do gov.br e posso te ...
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...,Olá! Sou o assistente do gov.br e posso te aju...
...,...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...,Olá! Eu sou o assistente digital do gov.br e p...
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...,Olá! Eu sou o assistente oficial do gov.br e p...
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....,Olá! Eu sou o assistente do gov.br e posso te ...
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...,"Olá! Sou o assistente oficial do gov.br, pront..."


In [3]:
# os.environ["OPENAI_API_KEY"] = ""

client = OpenAI()
model = "gpt-4-turbo"

In [4]:
def generate_prompt(pergunta, resposta, resposta_esperada="Não fornecido"):
    file_path = 'llm_judge_prompt.txt'

    with open(file_path, 'r') as file:
        file_content = file.read()

    prompt = file_content.replace(
        "[PERGUNTA]", pergunta
    ).replace("[RESPOSTA]", resposta).replace("[RESPOSTA_ESPERADA]", resposta_esperada)
    
    return prompt

In [5]:
# Create parser
parser = PydanticOutputParser(pydantic_object=CriteriaResponse)

# response test
# response = '{\n    "crit_1": 8, \n    "crit_2": 9, \n    "crit_3": 7, \n    "crit_4": 8, \n    "crit_5": 7, \n    "crit_6": 7, \n    "crit_7": 8, \n    "crit_8": 8\n}'

# try:
#     parsed_result = parser.parse(response)
#     print(parsed_result.crit_1)  # 8
#     print(parsed_result.dict())  # Convert to dict if needed
# except Exception as e:
#     print(f"Parsing failed: {e}")

In [6]:
data_for_evaluation[[
    "crit_1",
    "crit_2",
    "crit_3",
    "crit_4",
    "crit_5",
    "crit_6",
    "crit_7",
    "crit_8",
]] = "", "", "", "", "", "", "", "" 

for idx, obj in tqdm(data_for_evaluation.iterrows(), total=data_for_evaluation.shape[0]):
    prompt = generate_prompt(obj['pergunta'], obj['resposta'], obj['response_dify'])
    print("##########################################")
    print("processing question", idx)
    try:
        completion = client.chat.completions.create(
            model=model,
            messages=[
              {"role": "system", "content": "You are helpful AI assistant."},
              {"role": "user", "content": prompt}
            ]
        )
        text = completion.choices[0].message.content
        parsed_result = parser.parse(text)
        print(parsed_result.model_dump())
    except Exception as e:
        print(f"Parsing failed: {e}")
        continue
    data_for_evaluation.loc[idx, 'crit_1'] = parsed_result.crit_1
    data_for_evaluation.loc[idx, 'crit_2'] = parsed_result.crit_2
    data_for_evaluation.loc[idx, 'crit_3'] = parsed_result.crit_3
    data_for_evaluation.loc[idx, 'crit_4'] = parsed_result.crit_4
    data_for_evaluation.loc[idx, 'crit_5'] = parsed_result.crit_5
    data_for_evaluation.loc[idx, 'crit_6'] = parsed_result.crit_6
    data_for_evaluation.loc[idx, 'crit_7'] = parsed_result.crit_7
    data_for_evaluation.loc[idx, 'crit_8'] = parsed_result.crit_8

    # break

  0%|          | 0/79 [00:00<?, ?it/s]

##########################################
processing question 0
{'crit_1': 10, 'crit_2': 10, 'crit_3': 8, 'crit_4': 10, 'crit_5': 10, 'crit_6': 10, 'crit_7': 10, 'crit_8': 9}
##########################################
processing question 1
{'crit_1': 8, 'crit_2': 6, 'crit_3': 5, 'crit_4': 9, 'crit_5': 7, 'crit_6': 0, 'crit_7': 8, 'crit_8': 5}
##########################################
processing question 2
{'crit_1': 10, 'crit_2': 10, 'crit_3': 9, 'crit_4': 9, 'crit_5': 9, 'crit_6': -1, 'crit_7': 8, 'crit_8': 10}
##########################################
processing question 3
{'crit_1': 10, 'crit_2': 10, 'crit_3': 9, 'crit_4': 8, 'crit_5': 10, 'crit_6': 10, 'crit_7': 9, 'crit_8': 9}
##########################################
processing question 4
Parsing failed: Failed to parse CriteriaResponse from completion {"crit_1": 2, "crit_2": 1, "crit_3": 1, "crit_4": 4, "crit_5": 4, "crit_6": null, "crit_7": 2, "crit_8": 1}. Got: 1 validation error for CriteriaResponse
crit_6
  Input should 

In [7]:
data_for_evaluation

,pergunta,classe,resposta,response_dify,crit_1,crit_2,crit_3,crit_4,crit_5,crit_6,crit_7,crit_8
0,Quais serviços estão disponíveis nos balcões d...,Geral – Atendimento Presencial,A plataforma gov.br está com uma iniciativa de...,Olá! Sou o assistente do gov.br e estou aqui p...,10,10,8,10,10,10,10,9
1,Qual suporte vocês oferecem para problemas enc...,Geral - Problema em sistemas diversos,Escrevemos em atenção à sua solicitação para i...,Olá! Sou o assistente do gov.br e posso ajudar...,8,6,5,9,7,0,8,5
2,Como faço para obter um arquivo com o registro...,Geral - Arquivo da conversa no chat,"Para solicitar o arquivo com esta conversa, pe...",Olá! Sou o assistente do gov.br e posso te aju...,10,10,9,9,9,-1,8,10
3,Qual é o número de telefone para atendimento s...,Geral - Atendimento telefônico,Não há canal para atendimento telefônico sobre...,Olá! Eu sou o assistente do gov.br e posso te ...,10,10,9,8,10,10,9,9
4,Como posso deixar uma avaliação positiva sobre...,Geral - Elogios,Agradecemos o seu elogio! A equipe gov.br está...,Olá! Sou o assistente do gov.br e posso te aju...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
74,Como faço para acessar os serviços de Imposto ...,Nível insuficiente - Imposto de Renda,Os serviços de Imposto de Renda (https://www.g...,Olá! Eu sou o assistente digital do gov.br e p...,9,10,8,9,9,0,9,9
75,Como um MEI pode acessar e melhorar o nível da...,Nível insuficiente - MEI,Os serviços de MEI são prestados pela Receita ...,Olá! Eu sou o assistente oficial do gov.br e p...,9,9,8,7,8,7,7,9
76,Como posso acessar os serviços do Meu INSS se ...,Nível insuficiente - Meu INSS,Os serviços do Meu INSS (https://meu.inss.gov....,Olá! Eu sou o assistente do gov.br e posso te ...,9,10,9,8,8,-1,8,9
77,Como posso acessar e utilizar o sistema Regist...,Nível insuficiente - Registrato,O sistema Registrato (https://registrato.bcb.g...,"Olá! Sou o assistente oficial do gov.br, pront...",9,10,8,9,8,-1,8,9


In [8]:
data_for_evaluation.to_csv("dify-17.06.2025-evaluated.csv", index=False)